# 3D Spatial Image Filtering (Convolution)

## Implementing and Running Convolution "from scratch"

In [1]:
import SimpleITK as sitk
import numpy as np

def convolve_3d(image, kernel):
    # Get the dimensions of the image and the kernel
    image_array = sitk.GetArrayFromImage(image)
    kernel_array = np.array(kernel)
    
    image_shape = image_array.shape
    kernel_shape = kernel_array.shape
    
    # Define the padding size
    pad_size = tuple((ks // 2 for ks in kernel_shape))
    
    # Pad the image
    padded_image = np.pad(image_array, [(p, p) for p in pad_size], mode='constant', constant_values=0)
    
    # Prepare the output array
    output_array = np.zeros_like(image_array)
    
    # Perform the convolution
    for z in range(image_shape[0]):
        for y in range(image_shape[1]):
            for x in range(image_shape[2]):
                # Extract the region of interest
                region = padded_image[z:z + kernel_shape[0], y:y + kernel_shape[1], x:x + kernel_shape[2]]
                # Apply the convolution (element-wise multiplication and summation)
                output_array[z, y, x] = np.sum(region * kernel_array)
    
    # Convert the output array to a SimpleITK image
    output_image = sitk.GetImageFromArray(output_array)
    output_image.CopyInformation(image)
    
    return output_image

# Example usage
if __name__ == "__main__":
    # Load a 3D image
    image = sitk.ReadImage('data/I7025.nii.gz')
    
    # Define a 3D kernel (e.g., 3x3x3 mean filter)
    kernel = np.ones((3, 3, 3)) / 27
    
    # Apply the custom 3D convolution
    convolved_image = convolve_3d(image, kernel)
    
    # Save the convolved image
    sitk.WriteImage(convolved_image, 'convolved_3D_image_nparray.nii')
    
    print("3D convolution applied and saved as 'convolved_3D_image.nii'")


3D convolution applied and saved as 'convolved_3D_image.nii'


## Using a Convolution function from the SimpleITK package

In [5]:
import SimpleITK as sitk
import numpy as np

def convolve_3d_image(image, kernel):
    """
    Apply 3D convolution to a given image using a specified kernel.

    Parameters:
    - image (sitk.Image): The input 3D image.
    - kernel (np.ndarray): 3D kernel to be used for convolution.

    Returns:
    - convolved_image (sitk.Image): The convolved 3D image.
    """
    # Ensure the image is in float format 
    image = sitk.Cast(image, sitk.sitkFloat32)

    # Convert the kernel to a SimpleITK image
    kernel_image = sitk.GetImageFromArray(kernel.astype(np.float32))
    
    # Apply the convolution using sitk.Convolution
    convolved_image = sitk.Convolution(image, kernel_image)
    
    return convolved_image

# Example usage
if __name__ == "__main__":
    # Path to the input 3D image
    input_image_path = 'data/I7025.nii.gz'
    
    # Load the 3D image
    image = sitk.ReadImage(input_image_path)
    
    # Define a 3D kernel (e.g., 3x3x3 mean filter)
    kernel = np.ones((3, 3, 3)) / 27
    
    # Apply the 3D convolution

    # the sitk.Convolution function in SimpleITK handles padding internally. 
    # By default, it uses a boundary condition known as "zero-flux Neumann" 
    # (also called "symmetric" padding in other contexts), which ensures that 
    # the values at the border of the image are reflected. This helps to mitigate 
    # the border effects that occur during convolution.

    # Zero-Flux Neumann Padding
    # Description: The values at the edges of the image are reflected, meaning 
    # the border pixels are mirrored to extend the image.

    # Advantages: Preserves the continuity of the image at the borders and reduces 
    # edge artifacts.

    # Implementation: This is handled internally by the sitk.Convolution function, 
    # so you don't need to explicitly pad the image.

    # This internal handling of padding makes the sitk.Convolution function 
    # convenient and easy to use for applying convolutions without worrying about 
    # boundary conditions.

    convolved_image = convolve_3d_image(image, kernel)
    
    # Path to save the convolved 3D image
    output_image_path = 'convolved_3D_image_sitk.nii'
    
    # Save the convolved image
    sitk.WriteImage(convolved_image, output_image_path)
    
    print(f"3D convolution applied and saved as '{output_image_path}'")


3D convolution applied and saved as 'convolved_3D_image_sitk.nii'


## Using a Convolution function from the scipy.ndimage package

In [6]:
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import convolve

def convolve_3d_image_scipy(image, kernel):
    """
    Apply 3D convolution to a given image using a specified kernel with SciPy.

    Parameters:
    - image (sitk.Image): The input 3D image.
    - kernel (np.ndarray): 3D kernel to be used for convolution.

    Returns:
    - convolved_image (sitk.Image): The convolved 3D image.
    """
    # Convert the SimpleITK image to a NumPy array
    image_array = sitk.GetArrayFromImage(image)
    
    # Ensure the image and kernel are in float format
    image_array = image_array.astype(np.float32)
    kernel = kernel.astype(np.float32)
    
    # Apply the convolution using scipy.ndimage.convolve
    convolved_array = convolve(image_array, kernel, mode='constant', cval=0.0)
    
    # Convert the convolved array back to a SimpleITK image
    convolved_image = sitk.GetImageFromArray(convolved_array)
    convolved_image.CopyInformation(image)
    
    return convolved_image

# Example usage
if __name__ == "__main__":
    # Path to the input 3D image
    input_image_path = 'data/I7025.nii.gz'
    
    # Load the 3D image
    image = sitk.ReadImage(input_image_path)
    
    # Define a 3D kernel (e.g., 3x3x3 mean filter)
    kernel = np.ones((3, 3, 3)) / 27
    
    # Apply the 3D convolution
    convolved_image = convolve_3d_image_scipy(image, kernel)
    
    # Path to save the convolved 3D image
    output_image_path = 'convolved_3D_image_scipy.nii'
    
    # Save the convolved image
    sitk.WriteImage(convolved_image, output_image_path)
    
    print(f"3D convolution applied using SciPy and saved as '{output_image_path}'")


3D convolution applied using SciPy and saved as 'convolved_3D_image_scipy.nii'


## 3D Gaussian filtering using np.ndarray 

In [4]:
import numpy as np
import SimpleITK as sitk
from scipy.signal import convolve

def gaussian_kernel(size, sigma):
    """
    Generate a 3D Gaussian kernel.
    
    Parameters:
    - size (int): The size of the kernel (should be odd).
    - sigma (float): The standard deviation of the Gaussian distribution.
    
    Returns:
    - kernel (np.ndarray): The 3D Gaussian kernel.
    """
    kernel_range = np.linspace(-(size // 2), size // 2, size)
    x, y, z = np.meshgrid(kernel_range, kernel_range, kernel_range)
    kernel = np.exp(-(x**2 + y**2 + z**2) / (2 * sigma**2))
    kernel = kernel / np.sum(kernel)  # Normalize the kernel
    return kernel

def apply_gaussian_filter(image, kernel):
    """
    Apply a 3D Gaussian filter to a 3D image using convolution.
    
    Parameters:
    - image (np.ndarray): The 3D image.
    - kernel (np.ndarray): The 3D Gaussian kernel.
    
    Returns:
    - filtered_image (np.ndarray): The filtered 3D image.
    """
    filtered_image = convolve(image, kernel, mode='same')
    return filtered_image

# Example usage
if __name__ == "__main__":
    # Load the 3D image from a .nii file using SimpleITK
    image = sitk.ReadImage('data/I7025.nii.gz')
    image_array = sitk.GetArrayFromImage(image)
    
    # Generate a 3D Gaussian kernel
    size = 3  # Size of the kernel
    sigma = 1.0  # Standard deviation
    kernel = gaussian_kernel(size, sigma)
    
    # Apply the Gaussian filter
    filtered_image_array = apply_gaussian_filter(image_array, kernel)
    
    # Convert the filtered array back to a SimpleITK image
    filtered_image = sitk.GetImageFromArray(filtered_image_array)
    filtered_image.CopyInformation(image)
    
    # Save the filtered image to a .nii file
    sitk.WriteImage(filtered_image, 'convolved_3D_image_nparray.nii')
    
    print("Filtered image using NumPy saved to 'convolved_3D_image.nii'")


Filtered image using NumPy saved to 'convolved_3D_image.nii'


## 3D Gaussian filtering using SimpleITK     

In [ ]:
import SimpleITK as sitk

def apply_gaussian_filter_sitk(image_path, sigma, output_path):
    """
    Apply a Gaussian filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - sigma (float): The standard deviation of the Gaussian distribution.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Apply Gaussian filter
    gaussian_filter = sitk.SmoothingRecursiveGaussianImageFilter()
    gaussian_filter.SetSigma(sigma)
    filtered_image = gaussian_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    apply_gaussian_filter_sitk('data/I7025.nii.gz', 1.0, 'convolved_3D_image_sitk.nii')


Filtered image using SimpleITK saved to convolved_3D_image_sitk.nii


## 3D Gaussian filtering using scipy

In [12]:
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import gaussian_filter

def apply_gaussian_filter_scipy(image, sigma):
    """
    Apply a Gaussian filter to a 3D image using SciPy.
    
    Parameters:
    - image (np.ndarray): The 3D image.
    - sigma (float): The standard deviation of the Gaussian distribution.
    
    Returns:
    - filtered_image (np.ndarray): The filtered 3D image.
    """
    filtered_image = gaussian_filter(image, sigma=sigma)
    return filtered_image

# Example usage
if __name__ == "__main__":
    # Load the 3D image from a .nii file using SimpleITK
    image = sitk.ReadImage('data/I7025.nii.gz')
    image_array = sitk.GetArrayFromImage(image)
    
    # Apply the Gaussian filter
    sigma = 1.0  # Standard deviation
    filtered_image_array = apply_gaussian_filter_scipy(image_array, sigma)
    
    # Convert the filtered array back to a SimpleITK image
    filtered_image = sitk.GetImageFromArray(filtered_image_array)
    filtered_image.CopyInformation(image)
    
    # Save the filtered image to a .nii file
    sitk.WriteImage(filtered_image, 'convolved_3D_image_scipy.nii')
    
    print("Filtered image using SciPy saved to 'convolved_3D_image.nii'")


Filtered image using SciPy saved to 'convolved_3D_image.nii'


## Non-Linear Filtering - Median Filter using SimpleITK

In [11]:
import SimpleITK as sitk

def apply_median_filter_sitk(image_path, size, output_path):
    """
    Apply a 3D median filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - size (int): The size of the median filter kernel.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Apply Median filter
    median_filter = sitk.MedianImageFilter()
    median_filter.SetRadius(size)
    filtered_image = median_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    apply_median_filter_sitk('data/I7025.nii.gz', 1, 'medial_filtering_3D_image.nii')


Filtered image using SimpleITK saved to medial_filtering_3D_image.nii


## Non-Linear Filtering - Bilateral Filter using SimpleITK

In [17]:
import SimpleITK as sitk

def apply_bilateral_filter_sitk(image_path, domain_sigma, range_sigma, output_path):
    """
    Apply a 3D bilateral filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - domain_sigma (float): The standard deviation for the spatial Gaussian kernel.
    - range_sigma (float): The standard deviation for the range Gaussian kernel.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Apply Bilateral filter
    bilateral_filter = sitk.BilateralImageFilter()
    bilateral_filter.SetDomainSigma(domain_sigma)
    bilateral_filter.SetRangeSigma(range_sigma)
    filtered_image = bilateral_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    apply_bilateral_filter_sitk('data/I7025.nii.gz', 2.0, 50.0, 'bilateral_filtering_3D_image.nii')


Filtered image using SimpleITK saved to bilateral_filtering_3D_image.nii


## Non-Linear Filtering - Anisotropic Diffusion Filter using SimpleITK

In [19]:
import SimpleITK as sitk

def apply_anisotropic_diffusion_sitk(image_path, time_step, conductance, iterations, output_path):
    """
    Apply a 3D anisotropic diffusion filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - time_step (float): The time step for the diffusion process.
    - conductance (float): The conductance parameter for the diffusion process.
    - iterations (int): Number of iterations for the diffusion process.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Cast the image to a supported pixel type (float32)
    image = sitk.Cast(image, sitk.sitkFloat32)
    
    # Apply Anisotropic Diffusion filter
    anisotropic_diffusion_filter = sitk.CurvatureAnisotropicDiffusionImageFilter()
    anisotropic_diffusion_filter.SetTimeStep(time_step)
    anisotropic_diffusion_filter.SetConductanceParameter(conductance)
    anisotropic_diffusion_filter.SetNumberOfIterations(iterations)
    filtered_image = anisotropic_diffusion_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    apply_anisotropic_diffusion_sitk('data/I7025.nii.gz', 0.0625, 1.0, 10, 'anisotropic_diffusion_3D_image.nii')


Filtered image using SimpleITK saved to anisotropic_diffusion_3D_image.nii


## Non-Linear Filtering - NonLocal Means Filter using 

In [ ]:
# Need to install
# pip install scikit-image
# pip install imageio-ffmpeg (not sure if this is needed ???)
# pip install nibabel
# pip install PyWavelets

import numpy as np
from skimage.restoration import denoise_nl_means, estimate_sigma
import nibabel as nib

def apply_nonlocal_means_skimage(image_path, patch_size, patch_distance, h, fast_mode, output_path):
    """
    Apply a 3D Non-Local Means filter to a 3D image using scikit-image.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - patch_size (int): The size of the patches used for denoising.
    - patch_distance (int): The distance for the neighborhood search.
    - h (float): The filtering parameter.
    - fast_mode (bool): Whether to use the fast mode or not.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = nib.load(image_path)
    image_data = image.get_fdata()
    
    # Estimate the noise standard deviation from the noisy image
    sigma_est = np.mean(estimate_sigma(image_data))
    
    # Apply Non-Local Means filter
    denoised_image = denoise_nl_means(image_data, patch_size=patch_size, patch_distance=patch_distance, h=h*sigma_est, fast_mode=fast_mode)
    
    # Save the filtered image
    denoised_image_nib = nib.Nifti1Image(denoised_image, image.affine)
    nib.save(denoised_image_nib, output_path)
    print(f"Filtered image using scikit-image saved to {output_path}")

# Example usage
if __name__ == "__main__":
    apply_nonlocal_means_skimage('data/I7025.nii.gz', 3, 5, 1.15, True, 'nonlocal_means_3D_image_skimage.nii')


Filtered image using scikit-image saved to nonlocal_means_3D_image_skimage.nii


In [ ]:
import SimpleITK as sitk

def apply_nonlocal_means_sitk(image_path, patch_radius, output_path):
    """
    Apply a 3D Non-Local Means filter to a 3D image using SimpleITK.
    
    Parameters:
    - image_path (str): Path to the input 3D image.
    - patch_radius (int): The radius for the neighborhood search.
    - output_path (str): Path to save the filtered image.
    """
    # Load the image
    image = sitk.ReadImage(image_path)
    
    # Apply Non-Local Means filter
    nlm_filter = sitk.PatchBasedDenoisingImageFilter()
    nlm_filter.SetPatchRadius(patch_radius)
    nlm_filter.SetNoiseModel(sitk.PatchBasedDenoisingImageFilter.GAUSSIAN)
    filtered_image = nlm_filter.Execute(image)
    
    # Save the filtered image
    sitk.WriteImage(filtered_image, output_path)
    print(f"Filtered image using SimpleITK saved to {output_path}")

# Example usage
if __name__ == "__main__":
    apply_nonlocal_means_sitk('data/I7025.nii.gz', 4, 'nonlocal_means_3D_image.nii')
